# ch5. 오차역전파법

## 5.1 계산 그래프

계산그래프는 계산 과정을 그래프로 나타낸 것
계산그래프는 복수의 노드와 엣지로 표현된다.

* 국소적 계산:

계산 그래프의 특징은 국소적 계산을 전파함으로써 최종결과를 얻는다는 점에 있다.
자신과 관계된 정보만으로 결과를 출력한다.

## 5.2 연쇄법칙

역전파는 '국소적 미분'을 순방향의 반대방향으로 전달한다.
이 '국소적 미분'을 전달하는 원리가 <연쇄법칙>이다. 

### 연쇄법칙

합성함수란 여러 함수로 구성된 함수를 말한다.

연쇄법칙은 합성함수의 미분에 대한 성질이며,

합성 함수의 미분은 합성함수를 구성하는 각 함수의 미분의 곱으로 나타낼 수 있다는 것이 연쇄법칙의 원리이다.

## 5.3 역전파

계산그래프의 역전파는 연쇄법칙에 따라 국소적 미분이 전달되는 것.

- 덧셈 노드의 역전파는 상류의 값이 하류로 그대로 전달되고, 
- 곱셈 노드의 역전파는 순전파 때의 입력신호들을 서로 바꾼 값을 곱해서 하류로 전달된다.

## 5.4 단순한 계층 구현하기

In [3]:
#곱셈계층
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out

    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy

#덧셈계층
class AddLayer:
    def __init__(self):
        pass

    def forward(self, x, y):
        out = x + y
        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1
        return dx, dy

## 5.5 활성화 함수 계층 구현하기

- Relu 계층

In [4]:
class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0      #순전파 때의 입력이 0이하인 경우만 미분값 0(신호를 보내지 않음.)
        dx = dout

        return dx

- Sigmoid 계층

시그모이드 계층의 역전파는 순전파의 출력만으로 계산할 수 있다.

In [6]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out

        return out

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx

## 5.6 Affine/Softmax 계층 구현하기

- Affine 계층

신경망의 순전파 때 수행하는 행렬의 곱을 기하학에서는 '어파인 변환'이라고 한다.
어파인 변환을 수행하는 처리를 'Affine 계층'이라고 함.

In [7]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None

    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b

        return out

    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)

        return dx

- Softmax-with-loss 계층

신경망에서 수행하는 작업은 학습/ 추론.

추론할 때는 softmax 계층을 사용하지 않는다.

Softmax 앞의 Affine 계층의 출력을 점수(score)라고 한다.

In [9]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)  # 오버플로 대책
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a

    return y


def cross_entropy_error(y, t):
    delta = 1e-7  # 0일때 -무한대가 되지 않기 위해 작은 값을 더함
    return -np.sum(t * np.log(y + delta))


class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None  # 손실
        self.y = None     # softmax의 출력
        self.t = None     # 정답 레이블(원-핫 벡터)

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)  # 3.5.2, 4.2.2에서 구현
        self.loss = cross_entropy_error(self.y, self.t)

        return self.loss

    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = self.y - self.t / batch_size

        return dx

## 5.7 오차역전파법 구현하기

신경망 학습의 순서

1. 미니 배치 (train 데이터 중 일부를 가져옴, 미니배치(선별한 데이터셋)의 손실함수값을 줄이는 것이 목표)


2. 기울기 산출 (미니배치의 손실함수 값을 줄이기 위해 각 가중치 매개변수의 기울기르 구한다. 기울기는 손실함수의 값을 가장 작게 하는 방향을 제시)


3. 매개변수 갱신(가중치 매개변수를 기울기 방향으로 아주 조금씩 갱신함)


4. 1~3 단계를 반복한다.

-> 오차역전파법을 이용하면, 계산시간이 오래걸리는 수치미분과 달리 기울기를 효율적이고 빠르게 구할 수 있다.

In [15]:
import sys
import os
import numpy as np
from collections import OrderedDict
sys.path.append(os.pardir)
from common.layers import *
from common.gradient import numerical_gradient
from dataset.mnist import load_mnist

In [16]:
class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size,
        weight_init_std=0.01):
        # 가중치 초기화
        self.params = {}
        self.params['W1'] = weight_init_std * \
            np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * \
            np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

        # 계층 생성
        self.layers = OrderedDict()
        self.layers['Affine1'] = \
            Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = \
            Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()

    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    # x : 입력 데이터, t : 정답 레이블
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastLayer.forward(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1:
            t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])

        return grads

    def gradient(self, x, t):
        # 순전파
        self.loss(x, t)

        # 역전파
        dout = 1
        dout = self.lastLayer.backward(dout)

        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)

        # 결과 저장
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db

        return grads

- 오차역전파법을 통한 기울기 검증

수치미분의 결과와 오차역전파법의 결과를 비교해서 오차역전파법을 제대로 구현했는지 검증한다. -> 기울기 확인 작업

In [17]:
(x_train, t_train), (x_test, t_test) = \
        load_mnist(normalize=True, one_hot_label=True)

network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

# 각 가중치의 차이의 절댓값을 구한 후, 그 절댓값들의 평균을 낸다.
for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key] - grad_numerical[key]))
    print(key + ":" + str(diff))

W1:2.2193749612418504e-13
b1:6.687288309908945e-13
W2:8.339658866845959e-13
b2:1.2034817170603062e-10


### 5.7.4 오차역전파법을 사용한 학습 구현하기

In [19]:
import sys
import os
import numpy as np
sys.path.append(os.pardir)
from dataset.mnist import load_mnist
from two_layer_net import TwoLayerNet

# 데이터 읽기
(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

# 하이퍼 파라메터
iters_num = 10000  # 반복횟수
train_size = x_train.shape[0]
batch_size = 100  # 미니배치 크기
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

# 1에폭당 반복 수
iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    # print(i)
    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 오차역전파법으로 기울기 계산
    grad = network.gradient(x_batch, t_batch)

    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key]

    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    # 1에폭 당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc | " + str(train_acc) + ", " + str(test_acc))

train acc, test acc | 0.1222, 0.1129
train acc, test acc | 0.9009666666666667, 0.9069
train acc, test acc | 0.9224, 0.9241
train acc, test acc | 0.9322, 0.9342
train acc, test acc | 0.9421, 0.9389
train acc, test acc | 0.94895, 0.9451
train acc, test acc | 0.9558666666666666, 0.9515
train acc, test acc | 0.9612833333333334, 0.955
train acc, test acc | 0.9648, 0.9577
train acc, test acc | 0.9675833333333334, 0.9605
train acc, test acc | 0.9700666666666666, 0.9634
train acc, test acc | 0.9716, 0.9634
train acc, test acc | 0.9726833333333333, 0.9646
train acc, test acc | 0.9749333333333333, 0.9655
train acc, test acc | 0.9772666666666666, 0.9674
train acc, test acc | 0.97805, 0.9692
train acc, test acc | 0.9796166666666667, 0.9702
